In [1]:
import pandas as pd
import numpy as np
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("train.csv", encoding="utf-8")

In [4]:
df

,Cluster/Country/Region,SubCluster,calweek,brand,dc,product_code,customer_code,Quantity
0,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537
1,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544
2,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574
3,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523
4,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571
...,...,...,...,...,...,...,...,...
18463,US,SC2,202311,BRANDC,DC4,PRDCT9,CUSTZ,1753
18464,US,SC2,202312,BRANDC,DC4,PRDCT9,CUSTZ,1752
18465,US,SC2,202313,BRANDC,DC4,PRDCT9,CUSTZ,1673
18466,US,SC2,202314,BRANDC,DC4,PRDCT9,CUSTZ,1752


In [5]:
df.columns

Index(['Cluster/Country/Region', 'SubCluster', 'calweek', 'brand', 'dc',
       'product_code', 'customer_code', 'Quantity'],
      dtype='object')

In [6]:
df['Week'] = df['calweek'].astype(str).str[-2:].astype(int)

In [7]:
df['timeseries_key'] = df['product_code'] + '_' + df['customer_code']

In [8]:
df

,Cluster/Country/Region,SubCluster,calweek,brand,dc,product_code,customer_code,Quantity,Week,timeseries_key
0,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537,1,PRDCT1_CUSTA
1,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544,2,PRDCT1_CUSTA
2,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574,3,PRDCT1_CUSTA
3,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523,4,PRDCT1_CUSTA
4,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571,5,PRDCT1_CUSTA
...,...,...,...,...,...,...,...,...,...,...
18463,US,SC2,202311,BRANDC,DC4,PRDCT9,CUSTZ,1753,11,PRDCT9_CUSTZ
18464,US,SC2,202312,BRANDC,DC4,PRDCT9,CUSTZ,1752,12,PRDCT9_CUSTZ
18465,US,SC2,202313,BRANDC,DC4,PRDCT9,CUSTZ,1673,13,PRDCT9_CUSTZ
18466,US,SC2,202314,BRANDC,DC4,PRDCT9,CUSTZ,1752,14,PRDCT9_CUSTZ


In [9]:
df1 = df.drop(['Cluster/Country/Region', 'SubCluster', 'brand', 'dc'], axis = 1)

In [10]:
df1

,calweek,product_code,customer_code,Quantity,Week,timeseries_key
0,202001,PRDCT1,CUSTA,537,1,PRDCT1_CUSTA
1,202002,PRDCT1,CUSTA,544,2,PRDCT1_CUSTA
2,202003,PRDCT1,CUSTA,574,3,PRDCT1_CUSTA
3,202004,PRDCT1,CUSTA,523,4,PRDCT1_CUSTA
4,202005,PRDCT1,CUSTA,571,5,PRDCT1_CUSTA
...,...,...,...,...,...,...
18463,202311,PRDCT9,CUSTZ,1753,11,PRDCT9_CUSTZ
18464,202312,PRDCT9,CUSTZ,1752,12,PRDCT9_CUSTZ
18465,202313,PRDCT9,CUSTZ,1673,13,PRDCT9_CUSTZ
18466,202314,PRDCT9,CUSTZ,1752,14,PRDCT9_CUSTZ


In [11]:
df1['timeseries_key'] = df1['product_code'] + '_' + df1['customer_code']

In [12]:
timeseries_key_values = list(df1['timeseries_key'].unique())

### The Cluster/Country/Region, SubCluster, brand, dc, timeseries_key columns based on the Week number and timeseries_key from the week 16 to the week 28 for filling up the nan values.

In [13]:
start_week = 16
end_week = 28

filtered_df = df[(df['Week'] >= start_week) & (df['Week'] <= end_week)]

def generate_dataframes():
    grouped_df = filtered_df.groupby('timeseries_key')
    for key, group in grouped_df:
        group['timeseries_key'] = key
        yield group

combined_df = pd.concat(generate_dataframes(), ignore_index=True)
combined_df = combined_df.drop_duplicates(subset=['timeseries_key', 'Week'])
combined_df = combined_df.drop(["product_code", "customer_code", "Quantity", "Week", "calweek"], axis = 1)
combined_df

,Cluster/Country/Region,SubCluster,brand,dc,timeseries_key
0,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
1,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
2,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
3,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
4,US,SC1,BRANDA,DC1,PRDCT1_CUSTA
...,...,...,...,...,...
4181,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ
4182,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ
4183,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ
4184,US,SC2,BRANDC,DC4,PRDCT9_CUSTZ


In [14]:
# ARIMA model
def arima_forecast(train, test):
    
    # Find optimal p, d, q values using Auto ARIMA
    auto_arima_model = pm.auto_arima(train, seasonal=False, suppress_warnings=True)
    
    p, d, q = auto_arima_model.order
    model = ARIMA(train, order=(p, d, q))  # ARIMA(p, d, q)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# SARIMA model
def sarima_forecast(train, test):
    
    # Find optimal p, d, q, P, D, Q, and m values using Auto SARIMA
    auto_sarima_model = pm.auto_arima(train, seasonal=True, m=12, suppress_warnings=True)
    p, d, q = auto_sarima_model.order
    P, D, Q, m = auto_sarima_model.seasonal_order

    
    model = SARIMAX(train, order=(p, d, q), seasonal_order=(P, D, Q, m))  # SARIMA(p, d, q)(P, D, Q, m)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# Holt-Winters model
def holt_winters_forecast(train, test):
    model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add')
    model_fit = model.fit()
    forecast = model_fit.forecast(len(test))
    return forecast

# Facebook Prophet model
def prophet_forecast(train, test):
    model = Prophet()
    model.fit(train)
    future = model.make_future_dataframe(periods=len(test), freq='W')
    forecast = model.predict(future)
    forecast = forecast['yhat'][-len(test):].values
    return forecast


results_df = pd.DataFrame()
results_mape = pd.DataFrame()

In [15]:
  for i in timeseries_key_values:
    df_prod1 = df1[df1['timeseries_key']==i]
    
    print("----------RESULT FOR",i,"----------")
    
    # Convert 'calweek' column to datetime format
    df_prod1['calweek'] = pd.to_datetime(df_prod1['calweek'].astype(str) + '1', format='%Y%W%w')
    
    # Group data by 'calweek' and aggregate 'Quantity'
    ts_data = df_prod1.groupby('calweek')['Quantity'].sum().reset_index()
    
    # Rename columns for Prophet model
    ts_data.rename(columns={'calweek': 'ds', 'Quantity': 'y'}, inplace=True)

    # Split data into training and testing sets
    n_train = int(len(ts_data) * 0.8)  # 80% for training, 20% for testing
    train = ts_data.iloc[:n_train]
    test = ts_data.iloc[n_train:]
    
    # Perform forecasting using ARIMA
    arima_predictions = arima_forecast(train['y'], test['y'])

    # Perform forecasting using SARIMA
    sarima_predictions = sarima_forecast(train['y'], test['y'])

    # Perform forecasting using Holt-Winters
    holt_winters_predictions = holt_winters_forecast(train['y'], test['y'])
    

    # Perform forecasting using Facebook Prophet
    prophet_predictions = prophet_forecast(train, test)
    
    # Ensemble forecast combining ARIMA and Prophet
    ensemble_forecast = (arima_predictions + prophet_predictions) / 2.0

    # Calculate MAPE
    def calculate_mape(actual, forecast):
        return np.mean(np.abs((actual - forecast) / actual)) * 100

    # Calculate MAPE for ARIMA
    arima_mape = calculate_mape(test['y'], arima_predictions)

    # Calculate MAPE for SARIMA
    sarima_mape = calculate_mape(test['y'], sarima_predictions)

    # Calculate MAPE for Holt-Winters
    holt_winters_mape = calculate_mape(test['y'], holt_winters_predictions)

    # Calculate MAPE for Facebook Prophet
    prophet_mape = calculate_mape(test['y'], prophet_predictions)
    
    # Calculate MAPE for ensemble forecast
    ensemble_mape = calculate_mape(test['y'], ensemble_forecast)


    # Print MAPE values
    print("ARIMA - MAPE: {:.2f}%".format(arima_mape))
    print("SARIMA - MAPE: {:.2f}%".format(sarima_mape))
    print("Holt-Winters - MAPE: {:.2f}%".format(holt_winters_mape))
    print("Facebook Prophet - MAPE: {:.2f}%".format(prophet_mape))
    print("Ensemble Forecast - MAPE: {:.2f}%".format(ensemble_mape))
    
    #FORECASTINGGG
    
    # Split data into training and testing sets
    train = ts_data.iloc[:172]
    
    # Facebook Prophet model
    def prophet_forecast1(train):
        model = Prophet()
        model.fit(train)
        future = model.make_future_dataframe(periods=13, freq='W')
        forecast = model.predict(future)
        forecast = forecast['yhat'][-13:].values
        return forecast
    
    # ARIMA model
    def arima_forecast1(train):
        model = ARIMA(train, order=(1, 0, 0))  # ARIMA(p, d, q)
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=13)
        return forecast
    
    # Perform forecasting using ARIMA
    arima_predictions = arima_forecast1(train['y'])

    # Perform forecasting using Facebook Prophet
    prophet_predictions = prophet_forecast1(train)

    # Ensemble forecast combining ARIMA and Prophet
    ensemble_predictions = np.array((arima_predictions + prophet_predictions) / 2.0)
    print(ensemble_predictions)
    
    # Generate date values for the forecasted period
    start_date = ts_data['ds'].iloc[-1]
    forecast_dates = pd.date_range(start=start_date, periods=len(test), freq='W').strftime('%Y-%m-%d')
    for j in range(13):
        results_df = pd.concat([results_df, pd.DataFrame({
            'date': [forecast_dates[j]],
            'timeseries_key': [i],
            'Ensemble_Predicted': ensemble_predictions[j],
            'Prophet_Predicted': [prophet_predictions[j]]
            
        })], ignore_index=True)
        results_mape = pd.concat([results_mape, pd.DataFrame({
            'date': [forecast_dates[j]],
            'timeseries_key': [i],
            'Prophet_MAPE': [prophet_mape],
            'ARIMA_MAPE': [arima_mape],
            'SARIMA_MAPE': [sarima_mape],
            'HoltWinters_MAPE': [holt_winters_mape],
            'Ensemble_MAPE': [ensemble_mape]
        })], ignore_index=True) 

----------RESULT FOR PRDCT1_CUSTA ----------


11:48:18 - cmdstanpy - INFO - Chain [1] start processing
11:48:18 - cmdstanpy - INFO - Chain [1] done processing
11:48:18 - cmdstanpy - INFO - Chain [1] start processing


ARIMA - MAPE: 3.03%
SARIMA - MAPE: 3.03%
Holt-Winters - MAPE: 3.13%
Facebook Prophet - MAPE: 3.04%
Ensemble Forecast - MAPE: 2.96%


11:48:18 - cmdstanpy - INFO - Chain [1] done processing


[543.80917068 543.73081866 542.58441354 540.74138145 539.5042392
 539.93390737 541.89093775 544.00372798 544.5812084  542.7894989
 539.30904918 536.05424457 535.03774193]
----------RESULT FOR PRDCT1_CUSTB ----------


...

[1704.6568927  1706.61851051 1710.15500468 1709.64466061 1704.32156426
 1698.64658451 1698.067785   1703.92678813 1712.63891541 1719.2345535
 1721.08545355 1718.55865062 1713.37692869]
----------RESULT FOR PRDCT9_CUSTZ ----------


12:01:31 - cmdstanpy - INFO - Chain [1] start processing
12:01:31 - cmdstanpy - INFO - Chain [1] done processing
12:01:31 - cmdstanpy - INFO - Chain [1] start processing


ARIMA - MAPE: 2.04%
SARIMA - MAPE: 100.00%
Holt-Winters - MAPE: 1.97%
Facebook Prophet - MAPE: 2.10%
Ensemble Forecast - MAPE: 1.99%


12:01:31 - cmdstanpy - INFO - Chain [1] done processing


[1695.89060569 1697.86585517 1699.9130251  1696.14821817 1687.11464689
 1680.6289269  1683.81004709 1695.58144071 1707.4770173  1711.38302444
 1706.1346719  1697.22795765 1691.3318247 ]


In [16]:
results_df

,date,timeseries_key,Ensemble_Predicted,Prophet_Predicted
0,2023-04-16,PRDCT1_CUSTA,543.809171,539.394042
1,2023-04-23,PRDCT1_CUSTA,543.730819,539.376779
2,2023-04-30,PRDCT1_CUSTA,542.584414,537.079366
3,2023-05-07,PRDCT1_CUSTA,540.741381,533.393453
4,2023-05-14,PRDCT1_CUSTA,539.504239,530.919164
...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1707.477017,1716.515438
1400,2023-06-18,PRDCT9_CUSTZ,1711.383024,1724.327452
1401,2023-06-25,PRDCT9_CUSTZ,1706.134672,1713.830747
1402,2023-07-02,PRDCT9_CUSTZ,1697.227958,1696.017319


In [17]:
results_mape

,date,timeseries_key,Prophet_MAPE,ARIMA_MAPE,SARIMA_MAPE,HoltWinters_MAPE,Ensemble_MAPE
0,2023-04-16,PRDCT1_CUSTA,3.042337,3.026330,3.02633,3.133267,2.963487
1,2023-04-23,PRDCT1_CUSTA,3.042337,3.026330,3.02633,3.133267,2.963487
2,2023-04-30,PRDCT1_CUSTA,3.042337,3.026330,3.02633,3.133267,2.963487
3,2023-05-07,PRDCT1_CUSTA,3.042337,3.026330,3.02633,3.133267,2.963487
4,2023-05-14,PRDCT1_CUSTA,3.042337,3.026330,3.02633,3.133267,2.963487
...,...,...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,2.102834,2.043784,100.00000,1.973753,1.987289
1400,2023-06-18,PRDCT9_CUSTZ,2.102834,2.043784,100.00000,1.973753,1.987289
1401,2023-06-25,PRDCT9_CUSTZ,2.102834,2.043784,100.00000,1.973753,1.987289
1402,2023-07-02,PRDCT9_CUSTZ,2.102834,2.043784,100.00000,1.973753,1.987289


In [18]:
# Convert the date column to datetime format
results_df['date'] = pd.to_datetime(results_df['date'])

# Convert the date format to 'YYYY-WW'
results_df['calweek'] = results_df['date'].dt.strftime('%Y-%U')

In [19]:
results_df

,date,timeseries_key,Ensemble_Predicted,Prophet_Predicted,calweek
0,2023-04-16,PRDCT1_CUSTA,543.809171,539.394042,2023-16
1,2023-04-23,PRDCT1_CUSTA,543.730819,539.376779,2023-17
2,2023-04-30,PRDCT1_CUSTA,542.584414,537.079366,2023-18
3,2023-05-07,PRDCT1_CUSTA,540.741381,533.393453,2023-19
4,2023-05-14,PRDCT1_CUSTA,539.504239,530.919164,2023-20
...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1707.477017,1716.515438,2023-24
1400,2023-06-18,PRDCT9_CUSTZ,1711.383024,1724.327452,2023-25
1401,2023-06-25,PRDCT9_CUSTZ,1706.134672,1713.830747,2023-26
1402,2023-07-02,PRDCT9_CUSTZ,1697.227958,1696.017319,2023-27


In [20]:
merged_df = pd.merge(results_df, combined_df, on='timeseries_key')
merged_df.drop_duplicates(subset=['date', 'timeseries_key'], inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df

,date,timeseries_key,Ensemble_Predicted,Prophet_Predicted,calweek,Cluster/Country/Region,SubCluster,brand,dc
0,2023-04-16,PRDCT1_CUSTA,543.809171,539.394042,2023-16,US,SC1,BRANDA,DC1
1,2023-04-23,PRDCT1_CUSTA,543.730819,539.376779,2023-17,US,SC1,BRANDA,DC1
2,2023-04-30,PRDCT1_CUSTA,542.584414,537.079366,2023-18,US,SC1,BRANDA,DC1
3,2023-05-07,PRDCT1_CUSTA,540.741381,533.393453,2023-19,US,SC1,BRANDA,DC1
4,2023-05-14,PRDCT1_CUSTA,539.504239,530.919164,2023-20,US,SC1,BRANDA,DC1
...,...,...,...,...,...,...,...,...,...
1399,2023-06-11,PRDCT9_CUSTZ,1707.477017,1716.515438,2023-24,US,SC2,BRANDC,DC4
1400,2023-06-18,PRDCT9_CUSTZ,1711.383024,1724.327452,2023-25,US,SC2,BRANDC,DC4
1401,2023-06-25,PRDCT9_CUSTZ,1706.134672,1713.830747,2023-26,US,SC2,BRANDC,DC4
1402,2023-07-02,PRDCT9_CUSTZ,1697.227958,1696.017319,2023-27,US,SC2,BRANDC,DC4
